<div style="text-align: center; font-size:x-large;">기계학습 과제 3</div>

<div style="text-align: center; font-size:large;">2024. 12</div>

* 과제 2에서 선정한 데이터 및 목표를 가지고 부스팅 알고리즘 중 하나를 적용하여 그 코드와 결과를 보고하시오. 단, 최적의 학습 설정에 대해서는 CV를 사용하시오.

* 기말 발표에 대해서 아래 표를 점검하고 진행 완료된 것에 대하여 O를 표시해서 제출하시오.

&nbsp; 제출일 2024-12-12(목) 기준

|항목|진행여부|
|:-:|:-:|
|데이터 확정|O|
|데이터 설명(이름, 크기와 변수명, 목표)|O|
|데이터 시각화 및 EDA|O|
|일반 모형 적용(Linear Regression e.t.c.)|O|
|벌점 함수 적용(RIDGE or LASSO)|O|
|비선형 적용(GAM etc)|O|
|나무 모형(TREE)|O|
|부스팅(BOOSTING)|O|
|SVM||
|기타 비지도학습(추가)||

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

## **`XGBoost` 모형 적합**

&nbsp; 부스팅 모형 중 `XGBoost`를 이용하여 모형을 적합하였다. 부스팅 알고리즘은 변수 간 비선형 관계도 잘 설명할 수 있으므로, 표준화 등 변수의 변환은 수행하지 않고, 문자열로 입력된 거래일자 `date` 변수만 수치형으로 변환한 뒤 그대로 분석하였다.

## **1. imports**

---

In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import xgboost as xgb

from skopt import BayesSearchCV

warnings.filterwarnings("ignore", category=UserWarning)

## **2. Data**

---

In [2]:
## 원본 데이터
housing_price = pd.read_csv("./기말 발표/kc_house_data.csv")

## date 열 처리
df_preprocessed = housing_price.drop(["id", "date", "price"], axis = 1)\
.assign(date = pd.to_datetime(housing_price.date)).assign(price = housing_price.price)

## train/test set 분리
df_train, df_test = train_test_split(df_preprocessed, test_size = 0.3,
                                     shuffle = True, random_state = 14107)
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

## date 열 수치형으로 처리
df_train.date = df_train.date.map(lambda x : x - df_preprocessed.date.min()).dt.days
df_test.date = df_test.date.map(lambda x : x - df_preprocessed.date.min()).dt.days

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

## **3. 하이퍼파라미터 튜닝**

---

### **A. 서치 및 결과 (1)**

&nbsp; 부스팅 모형에선 고려해야 할 초매개변수가 상당히 많으므로, `skopt` 모듈의 `BayesSearchCV`를 사용하여 최적의 학습 설정을 찾아내고자 하였다. CV 기준값은 `rmse`를 사용하였으며, 4개의 폴드로 나누어 산출하였다.

&nbsp; 튜닝한 하이퍼파라미터는 `learning_rate`, `n_estimators`, `gamma`, `max_depth`, `min_child_weight`, `colsample_bytree`, `subsample`, `sampling_method`로 총 8개로 구성된다. 탐색 공간은 아래 표와 같은 범위로 설정하였으며, 이 중 노드 분할 시 최소 손실 감소량과 최소 헤시안 값을 나타내는 `gamma`와, `min_child_weight`의 경우 예측변수의 스케일이 백만 단위에 가깝기 때문에 그 값도 같이 커질 수 있다. 따라서 로그 스케일로 값을 선택하는 것이 바람직하다고 판단했다.

<div style="text-align: center; font-size:medium;"><하이퍼 파라미터 탐색 공간의 범위></div>

|hyperparameter|searching space|
|:-:|:-:|
|learning_rate|(0.001, 0.1)|
|n_estimators|(100, 10000)|
|gamma|(1e-4, 1000)|
|max_depth|(1, 15)|
|min_child_weight|(1e-2, 100)|
|colsample_bytree|(0.5, 1.0)|
|subsample|(0.5, 1.0)|
|sampling_method|[uniform, gradient_based]|

In [ ]:
## data
X = df_train.drop("price", axis = 1)
y = df_train.price

XX = df_test.drop("price", axis = 1)
yy = df_test.price

## params
fit_params = {
    'early_stopping_rounds': 10, ## 성능 개선이 없을 시 종료할 반복 수
    'verbose': False
}

search_space = {
    'learning_rate': (0.001, 0.1), ## 학습률
    'n_estimators': (100, 1000), ## 트리 수
    'gamma': (1e-4, 10000, "log-uniform"), ## 노드 분할 시 최소 손실 감소량
    'max_depth': (1, 15), ## 트리 깊이
    'min_child_weight': (1e-2, 100, "log-uniform"), ## 헤시안의 최소값
    'colsample_bytree': (0.5, 1.0), ## 각 트리 구성 시 사용하는 열의 비율
    'subsample': (0.5, 1.0), ## 데이터 서브 샘플링
    'sampling_method': ["uniform", "gradient_based"] ## 서브 샘플링 방법
}

kfold = KFold(4, random_state = 14107, shuffle = True)

## optimizr
predictr = xgb.XGBRegressor(tree_method = "gpu_hist", device = "cuda")
optimizr = BayesSearchCV(
    estimator=predictr,
    search_spaces=search_space,
    fit_params=fit_params,
    cv=kfold,
    scoring="neg_mean_squared_error",
    random_state=14107,
    verbose=1
)

In [ ]:
optimizr.fit(X, y)

In [ ]:
# optimizr.best_estimator_.get_params() ## 최적의 하이퍼 파라미터 값 딕셔너리

best_predictr = optimizr.best_estimator_
rmse = np.mean((yy - best_predictr.predict(XX))**2)**0.5
print(f"RMSE = {rmse:.4f}")

null_predictr = xgb.XGBRegressor()
null_predictr.fit(X, y)
rmse = np.mean((yy - null_predictr.predict(XX))**2)**0.5
print(f"RMSE = {rmse:.4f}")

RMSE = 108498.1906
RMSE = 114593.1858


&nbsp; 해당 방법으로 찾아낸 최적의 하이퍼 파라미터는 아래 표로 정리되었다. 이 때 test dataset에서 산출한 RMSE는 $108498.1906$이며, 해당 값은 모든 하이퍼파라미터를 디폴트로 놓고 적합한 모형의 RMSE $= 114593.1858$보다 작았다.

<div style="text-align: center; font-size:medium;"><최적의 하이퍼 파라미터(잠정)></div>
    
|hyperparameter|optimal value|
|:-:|:-:|
|learning_rate|0.0895|
|n_estimators|801|
|gamma|10000|
|max_depth|4|
|min_child_weight|0.9920|
|colsample_bytree|0.5|
|subsample|0.5|
|sampling_method|`gradient_based`|


> 하지만, 여기서 `gamma`와 `colsample_bytree`의 경우, 각각 범위의 최대값과 최소값으로 선택되어 탐색 공간이 최적점을 포함하지 못할 가능성이 존재한다. 따라서 범위를 조금 조정하여 한번 더 하이퍼파라미터를 튜닝하였다.

&nbsp;

### **B. 서치 및 결과 (2)**

In [3]:
## data
X = df_train.drop("price", axis = 1)
y = df_train.price

XX = df_test.drop("price", axis = 1)
yy = df_test.price

## params
fit_params = {
    'early_stopping_rounds': 10, ## 성능 개선이 없을 시 종료 라운드
    'verbose': False
}

search_space = {
    'learning_rate': (0.001, 0.1), ## 학습률
    'n_estimators': (100, 1000), ## 트리 수
    'gamma': (1e-1, 1e+6, "log-uniform"), ## 노드 분할 시 최소 손실 감소량
    'max_depth': (1, 10), ## 트리 깊이
    'min_child_weight': (1e-2, 100, "log-uniform"), ## 헤시안의 최소값
    'colsample_bytree': (0.2, 0.7), ## 각 트리 구성 시 사용하는 열의 비율
    'subsample': (0.5, 1.0), ## 데이터 서브 샘플링
    'sampling_method': ["uniform", "gradient_based"] ## 서브 샘플링 방법
}

kfold = KFold(4, random_state = 14107, shuffle = True)

## optimizr
predictr2 = xgb.XGBRegressor(tree_method = "gpu_hist", device = "cuda")
optimizr2 = BayesSearchCV(
    estimator=predictr2,
    search_spaces=search_space,
    fit_params=fit_params,
    cv=kfold,
    scoring="neg_mean_squared_error",
    random_state=14107,
    verbose=1
)

In [ ]:
optimizr2.fit(X, y)

In [7]:
# optimizr2.best_estimator_.get_params() ## 최적의 하이퍼 파라미터 값 딕셔너리

best_predictr2 = optimizr2.best_estimator_
rmse = np.mean((yy - best_predictr2.predict(XX))**2)**0.5
print(f"RMSE = {rmse:.4f}")

RMSE = 107775.8804


&nbsp; 최종 결정된 최적의 하이퍼 파라미터는 아래 표로 정리되었다. 이 때 test dataset에서 산출한 RMSE는 $107775.8804$으로, 이전에 교차검증으로 선택된 모형보다 더 좋은 성능지표를 보였다. 따라서 해당 모형을 예측 성능 관점에서 최선의 모형으로 선택한다.

&nbsp;

&nbsp;

<div style="text-align: center; font-size:medium;"><최종 결정된 하이퍼 파라미터></div>

|hyperparameter|optimal value|
|:-:|:-:|
|learning_rate|0.0940|
|n_estimators|1000|
|gamma|9571.96|
|max_depth|4|
|min_child_weight|1.7378|
|colsample_bytree|0.4693|
|subsample|0.5|
|sampling_method|`gradient_based`|